In [2]:
%load_ext autoreload
%autoreload

import dut_func as dut
import serial
import time
import struct
import numpy as np

from misc import *

# dut.drv.serUSB = 'COM19'
drv = dut.drv

ser = serial.Serial('COM6', 9600, timeout=1)
dut.connect(ser)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
dut.disconnect()

Serial port disconnected.


In [2]:
dut.pads_defaults()

In [3]:
dut.dac_set('PLANE_VPP', 0.22)

DAC initialized to a span from -5 V to 5 V


In [3]:
voltages = drv.pic_adc_read() *2
voltages

array([0.        , 0.07413919, 0.07575092, 0.50124542, 5.14461538])

Set up various DAC channels

In [3]:
dut.dac_set('DAC_VREF_HI_CMP', 3.92)

DAC initialized to a span from -5 V to 5 V


In [4]:
dut.dac_set('P_VREF_SH', 2.42)

In [5]:
dut.dac_set('P_VAGC_1', 3.9)

In [6]:
dut.dac_set('P_VAGC_0', 1)

In [7]:
dut.dac_set('P_TVDD', 4.92)

In [8]:
dut.dac_set('P_TVDD', 1.)

In [9]:
dut.dac_set('P_VREF_TIA', 0.42)

In [10]:
dut.dac_set('DAC_VREF_ARRAY', 0.42)

In [11]:
dut.dac_set('DAC_VP_PAD', 2.5)

In [12]:
dut.dac_set('P_AMP_VREF', 2.5)

In [13]:
dut.dac_set('P_AMP_INPUT', 2.5)

________________

Assert Power GOOD

In [14]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

Pulse RESET chip with a rising edge

In [15]:
portName, pinPos = PIC_PINS['NRESET_FULL_CHIP']
drv.gpio_pin_reset(portName, pinPos)
time.sleep(0.1) # want to delay 1us
drv.gpio_pin_set(portName, pinPos)

Send a scan chain data

In [16]:
data = bytes([0x10, 0x20, 0x0c, 0x10, 0x20, 0x01, 0x02])
drv.spi_serial_write(2, data)

In [18]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'


In [173]:
for _ in range(20):
    data = bytes([0x10, 0x20, 0x0c, 0x00, 0x20, 0x01, 0x02] * 20)
    drv.spi_serial_write(1, data)

In [89]:
data

b'\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02'

In [20]:
dut.disconnect()

Serial port disconnected.


In [242]:
# NOTE: first power on requires physical switch actions on DUT board as well as PIC commands

# FIRST: SWITCH microntroller VDD power ON (i.e. power up microcontroller)
time.sleep(1) # want to delay 1us
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
#dut.dac_init()

DAC initialized to a span from -5 V to 5 V


In [19]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [4]:
drv.clk_start('CK_ARRAY')
drv.clk_start('ADC_CK')

In [24]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
time.sleep(1e-6) # want to delay 1us
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [86]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [87]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [86]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [80]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

In [85]:
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE2'])

In [25]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [26]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])

In [27]:
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
# dac_set('P_ADC_EXT_TEST_IN', 1.5)       # voltage = 1.5 as an example, I wonder whether we should put it as a parameter of the function
time.sleep(1e-6)        # delay(t_sel_ext or t_ext_inp), min = 2CK
drv.gpio_pin_set(*PIC_PINS['ADC_SEL_EXTERNAL'])
#time.sleep(1e-6)       # delay(t_en_overide_sh), min = 0CK
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])
time.sleep(1e-6)        # delay(t_fire_sh), min = 3CK
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])
time.sleep(1e-6)        # delay(3 P_ADC_CK periods)
# drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])
while True:
    if drv.gpio_pin_is_high(*PIC_PINS['ADC_DONE']):
        break
#download_fifo()
dut.pads_defaults()
#reset_dpe()

KeyboardInterrupt: 

In [32]:
dut.pads_defaults()

In [29]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])

In [46]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])

In [31]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

WEDNESDAY ADC DONE TEST AND DOCUMENTATION

EXECUTE IN ORDER

In [5]:
dut.dac_init()

DAC initialized to a span from -5 V to 5 V


In [234]:
drv.gpio_pin_reset(*PIC_PINS['PWR_GOOD'])

In [235]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

In [236]:
dut.dac_set('PLANE_VPP', 0.22)

In [237]:
dut.dac_set('DAC_VREF_HI_CMP', 3.92)

In [238]:
dut.dac_set('DAC_VREF_ARRAY', 0.42)

In [239]:
dut.dac_set('P_VREF_SH', 2.42)

In [240]:
dut.dac_set('DAC_VP_PAD', 2.42)

In [241]:
dut.dac_set('P_VAGC_1', 3.9)

In [242]:
dut.dac_set('P_VAGC_0', 1)

In [243]:
dut.dac_set('P_TVDD', 4.92)

In [244]:
dut.dac_set('P_VREF_TIA', 0.42)

In [245]:
dut.dac_set('P_ADC_EXT_TEST_IN', 2.5)

FOR ADC OPERATION, PROCEED FROM HERE (07/24/2019) 16:40pm

In [259]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()

In [247]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])


In [248]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [249]:
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [250]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'
b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'


In [251]:
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])

In [252]:
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

In [253]:
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<2>'])

In [254]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [255]:
drv.gpio_pin_set(*PIC_PINS['ADC_SEL_EXTERNAL'])

In [256]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [257]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [180]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])

In [39]:
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE2'])

In [38]:
dut.pads_defaults()

In [182]:
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

BEGIN FIFO TESTING

In [223]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])


In [230]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])

In [231]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])

LOAD VECTORS

In [232]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])

In [233]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])

In [129]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [130]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

In [131]:
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [138]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'
b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'


In [ ]:
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])

In [ ]:
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

In [139]:
dut.pads_defaults()

In [156]:
roc = [0 ,0, 0]
# data= 0b1111111111111111
data= 0xffff
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [157]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0xffff
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

READ TESTING


In [139]:
dut.pads_defaults()

In [132]:
drv.gpio_pin_set(*PIC_PINS['DPE_INTERNAL_EN'])

In [133]:
drv.gpio_pin_set(*PIC_PINS['READ_DPE'])

In [134]:
drv.gpio_pin_set(*PIC_PINS['READ_BIT'])

In [ ]:
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<2>'])

In [135]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [137]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])

In [140]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_COLUMN_T'])

In [141]:
drv.gpio_pin_set(*PIC_PINS['DPE_PULSE'])

In [ ]:
drv.gpio_pin_reset(*PIC_PINS['DPE_PULSE'])